# Crash Analysis  
Due to time pressure crash analysis was done partly automated, partly by hand. It was mainly done using afl-cmin and crashwalk.

In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
import subprocess
import requests
from tempfile import TemporaryDirectory

## AFL-CMIN Validation

In [33]:
program_arguments = {
    'objdump'   : ['/home/deif/gits/fuzztestbench/objdumppre/objdump', '-D','@@']
,    'nm'       : ['/home/deif/gits/fuzztestbench/nmpre/nm-new', '-C','@@']
,    'ffmpeg'   : ['/home/deif/gits/fuzztestbench/ffmpegpre/ffmpeg','-i','@@']
,    'gif2png'  : ['/home/deif/gits/fuzztestbench/gif2pngpre/gif2png','@@']
,    'mupdf'    : ['/home/deif/gits/fuzztestbench/mupdfpre/mutool','show','@@']
,    'size'     : ['/home/deif/gits/fuzztestbench/sizepre/size','@@']
,    'tcpdump'  : ['/home/deif/gits/fuzztestbench/tcpdumppre/tcpdump', '-nr','@@']
,    'readelf'  : ['/home/deif/gits/fuzztestbench/readelfpre/readelf','-a','@@']
}

rodeo_arguments = {
    'bzip2': ['/home/deif/gits/fuzztestbench/bzip2pre/bzip2', '-c','@@'],
    'grep': ['/home/deif/gits/fuzztestbench/greppre/grep', '-f','@@'],
    'memdjpeg': ['/home/deif/gits/fuzztestbench/memdjpegpre/memdjpeg','@@'],
    'newgrep': ['/home/deif/gits/fuzztestbench/newgreppre/newgrep','-f','@@'],
}

base_folder = '/home/deif/gits/fuzztestbench/'
base_folder_neuzz = '/home/deif/gits/neuzz/programs/'


In [63]:
program = 'nm'
program_call = program_arguments[program]
program_call[0] = base_folder + program + 'pre/' + program_call[0]
fuzzer = 'curiousmutation'

In [36]:
crashes = {
    'program': [],
    'fuzzer': [],
    'crashes': []
}
fuzzers = ['afl', 'aflfast', 'curiousmutation', 'neuzz']
rodeo = True
if rodeo:
    prog_dict = rodeo_arguments
    range_max = 3
    mode = 'artificial'
else:
    prog_dict = program_arguments
    range_max = 6
    mode = 'real'

for program in prog_dict:
    
    program_call = prog_dict[program]

    for fuzzer in fuzzers:
        for i in range(1, range_max):
            with TemporaryDirectory(dir='./') as temp_dir: #create temp outputdir for every run
                if fuzzer != 'neuzz':
                    indir = os.path.join(base_folder, program + 'pre/' + program + 'pre' + fuzzer + str(i) + '/afl_out/crashes/')
                else:
                    indir = os.path.join(base_folder_neuzz, program + str(i) + '/crashes/')

                #os.path.isdir(indir) and
                if  len(os.listdir(indir))>0:
                    print(['afl-cmin', '-C', '-i', indir] + ['-o', temp_dir + '/'] + program_call)
                    out = subprocess.run(['afl-cmin', '-C', '-i', indir] + ['-o', temp_dir] + program_call)
                    #print(os.listdir(temp_dir))
                    crashes['program'].append(program)
                    crashes['fuzzer'].append(fuzzer)
                    crashes['crashes'].append(len(os.listdir(temp_dir)))
                else:
                    crashes['program'].append(program)
                    crashes['fuzzer'].append(fuzzer)
                    crashes['crashes'].append(0)

df_crashes = pd.DataFrame(crashes)
df_crashes.to_csv('./crashes_'+mode+'_programs.csv')

['afl-cmin', '-C', '-i', '/home/deif/gits/fuzztestbench/greppre/greppreafl1/afl_out/crashes/', '-o', './tmpv2fqbflk/', '/home/deif/gits/fuzztestbench/greppre/grep', '-f', '@@']
['afl-cmin', '-C', '-i', '/home/deif/gits/fuzztestbench/greppre/greppreafl2/afl_out/crashes/', '-o', './tmp0wos7j0p/', '/home/deif/gits/fuzztestbench/greppre/grep', '-f', '@@']
['afl-cmin', '-C', '-i', '/home/deif/gits/fuzztestbench/greppre/grepprecuriousmutation2/afl_out/crashes/', '-o', './tmp76xwutk7/', '/home/deif/gits/fuzztestbench/greppre/grep', '-f', '@@']
['afl-cmin', '-C', '-i', '/home/deif/gits/fuzztestbench/memdjpegpre/memdjpegpreafl1/afl_out/crashes/', '-o', './tmp4ub_ygos/', '/home/deif/gits/fuzztestbench/memdjpegpre/memdjpeg', '@@']
['afl-cmin', '-C', '-i', '/home/deif/gits/fuzztestbench/memdjpegpre/memdjpegpreafl2/afl_out/crashes/', '-o', './tmpsw3349hr/', '/home/deif/gits/fuzztestbench/memdjpegpre/memdjpeg', '@@']
['afl-cmin', '-C', '-i', '/home/deif/gits/fuzztestbench/memdjpegpre/memdjpegpreaflf

### real programs

In [39]:
df_crashes = pd.DataFrame(crashes)
for program in df_crashes['program'].unique():
    afl_mean = df_crashes.loc[(df_crashes.program==program) & (df_crashes.fuzzer=='afl')]['crashes'].mean()
    aflfast_mean = df_crashes.loc[(df_crashes.program==program) & (df_crashes.fuzzer=='aflfast')]['crashes'].mean()
    curiousafl_mean = df_crashes.loc[(df_crashes.program==program) & (df_crashes.fuzzer=='curiousmutation')]['crashes'].mean()
    neuzz_mean = df_crashes.loc[(df_crashes.program==program) & (df_crashes.fuzzer=='neuzz')]['crashes'].mean()

    if not (afl_mean == 0.0 and aflfast_mean == 0.0 and curiousafl_mean == 0.0 and neuzz_mean == 0.0):
        print(program + '\t' + 'afl: ' + str(afl_mean)+ '\t' + 'aflfast: ' + str(aflfast_mean)+ '\t' + 'curiousafl: ' + str(curiousafl_mean)+' \t' + 'neuzz: ' + str(neuzz_mean) )

memdjpeg	afl: 5.5	aflfast: 13.0	curiousafl: 15.5 	neuzz: 0.0
newgrep	afl: 67.5	aflfast: 67.0	curiousafl: 0.0 	neuzz: 14.5


## Real programs validation

In [47]:
program_arguments = {
    'objdump'   : ['objdump', '-D']
,    'nm'       : ['nm', '-C']
,    'ffmpeg'   : ['ffmpeg','-i']
,    'gif2png'  : ['gif2png', '']
,    'mupdf'    : ['mutool','show']
,    'size'     : ['size']
,    'tcpdump'  : ['tcpdump', '-nr'] 
,    'readelf'  : ['readelf','-a']
}

In [52]:
program = 'ffmpeg'
program_argument = ['~/gits/fuzztestbench/ffmpegpre/ffmpeg', '-i']#program_arguments[program]

for i in range(1,3):
    crash_path = '/home/deif/gits/fuzztestbench/' + program + 'pre/' + program + 'precuriousrandom10' + str(i) + '/'
    for filename in sorted(os.listdir(crash_path + 'afl_out/crashes/')):
        out = subprocess.run(program_argument + [os.path.join(crash_path + 'afl_out/crashes/', filename)])
        if out.returncode != 0:
            print(out)

FileNotFoundError: [Errno 2] No such file or directory: '~/gits/fuzztestbench/ffmpegpre/ffmpeg': '~/gits/fuzztestbench/ffmpegpre/ffmpeg'